# SISTEM IDENTIFIKASI SENTIMEN DATA TEKS MENGGUNAKAN METODE SUPPORT VECTOR MACHINE (SVM)

Yan Pieter Israel Rumere		(1808561082)

Ratri Desy Christirahma		(2108561003)

Saifulloh Rahman			(2108561028)

Nyoman Krisna Ari Sudarsana	( 2108561072)


# Data Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab-Notebooks/final-project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab-Notebooks/final-project


# Library

In [1]:
# Library untuk SVM
from flask import Flask, render_template, url_for
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import accuracy_score

# Packages untuk visuals
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(font_scale=1.2)

# Library/packages untuk preprocessing
import re
!pip install Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.9 MB/s eta 0:00:00


# Import Data

In [3]:
# load dataset dari gdrive
reviews = pd.read_csv("reviews.csv")

# menampilkan data
reviews

,No,Reviews,Label
0,1,kemeja nya bagusss bgtttt😍😍😍aaaa mauuu nngisss...,1
1,2,"Jahitannya sih rapi,cuman ada benang yang ikut...",0
2,3,Sesuai harga. Agak tipis tapi masih oke kok. W...,0
3,4,"Wah gila sihhh sebagus itu, se worth it, se l...",1
4,5,Kain nya bagus halus \nTapi kok di bukak koto...,0
...,...,...,...
826,827,Terima kasih barang sudah sampai sesuai ukuran...,1
827,828,Mantapp realpicttt bangttt tapi pengemasan nya...,1
828,829,"Suka bgt sama tasnya, ga kayak tas local. Kere...",1
829,830,kualitas produk sangat baik. produk original. ...,1


# Preprocessing

In [ ]:
# Preprpcessing
# Membuat objek stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Membuat objek stopword remover
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

# Lowercase
reviews['Reviews'] = reviews['Reviews'].apply(lambda x: x.lower())

# Removing new line (\n)
reviews['Reviews'] = reviews['Reviews'].apply(lambda x: x.replace('\n', ''))

# Removing Non-Alphabetic Characters
reviews['Reviews'] = reviews['Reviews'].apply(lambda x: re.compile('[^a-zA-Z]').sub(" ", x))

# Tokenisasi
reviews['Reviews'] = reviews['Reviews'].apply(lambda x: x.split())

# Removing Last Character from Each Word:
def removeAkhir(s):
  cadangan = s[-1]
  s = s.rstrip(s[-1])
  return s + cadangan
reviews['Reviews'] = reviews['Reviews'].apply(lambda x: [removeAkhir(i) for i in x])

# Remove short word
reviews['Reviews'] = reviews['Reviews'].apply(lambda x: [i for i in x if len(i) > 2])

# Joining Tokens into a Sentence
reviews['Reviews'] = reviews['Reviews'].apply(lambda x: " ".join(x))

# Stop word removal
reviews['Reviews'] = reviews['Reviews'].apply(lambda x: stopword.remove(x))

# Stemming
reviews['Reviews'] = reviews['Reviews'].apply(lambda x: stemmer.stem(x))

In [86]:
# reviews.to_csv(r"/content/drive/MyDrive/Colab-Notebooks/final-project/preprocessed-data.csv")

In [46]:
# data yg sudah di preprocessing
reviews = pd.read_csv("preprocessed-data.csv")

In [47]:
reviews

,Unnamed: 0,No,Reviews,Label
0,0,1,kemeja nya bagus bgt mau nngis knpa dri dlu be...,1
1,1,2,jahit sih rapi cuman benang ikut jahit jadi jelek,0
2,2,3,sesuai harga tipis masih oke kok warna abu kal...,0
3,3,4,wah gila sih bagus worth lembut baju kirain ba...,1
4,4,5,kain nya bagus halus kok bukak kotor warna putih,0
...,...,...,...,...
826,826,827,terima kasih barang sampai sesuai ukur seesuai...,1
827,827,828,mantap realpict bangt emas nya cuman plastik a...,1
828,828,829,suka bgt sama tas kayak tas local keren parah ...,1
829,829,830,kualitas produk sangat baik produk original ha...,1


# Split the data into training and test sets

In [49]:
# Spliting 80% train & 20% test, random_state=42 --> konsisten
train_X, test_X, train_Y, test_Y = model_selection.train_test_split(reviews['Reviews'], reviews['Label'], test_size = 0.2, random_state = 42)

In [50]:
# Membuat data frame untuk set data train
df_train80 = pd.DataFrame()
df_train80['Reviews'] = train_X
df_train80['Label'] = train_Y

# membuat data frame untuk set data test
df_test20 = pd.DataFrame()
df_test20['Reviews'] = test_X
df_test20['Label'] = test_Y

In [111]:
# menampilkan data train
df_train80

,Reviews,Label
477,gak pernah kecewa pokok nyaselalu bagus kirim ...,1
346,jual teliti barang kirim aju dana sulit dana h...,0
462,bagus cacat,1
670,shoppe mal embel doang kualitas apa kek baju p...,0
302,kirim nya lama banget terus udah smpei brng ny...,0
...,...,...
71,baju yang robek fatal bgt sumpah kecewa parah ...,0
106,plis inimah nyesel beli doang bahan adem pis n...,1
270,knp malah kirimin rusak sikecewa deh beli bara...,0
435,gak usah ragu beli tas disinikualitasnya buagu...,1


In [108]:
# menampilkan data test
df_test20

,Reviews,Label
610,kecewa sih sen nya warna hitam pas dateng nya ...,0
818,produk bagus sesuai sama gambar banget gitu ha...,1
290,bagus cuman susah atur jam nya susah tekan tekan,0
559,warna beda jauh digmbr jatuh nya wrnanya gelap,0
168,bagus kacamata mewah kayak mahal banget tebel ...,1
...,...,...
192,bagus cuma kirim lama bagus,1
650,kecewa ukur kaya ukur kecil bgt bahan tipis st...,0
456,jual gak amanah pesan datang cuma responnya la...,0
773,bagus,1


In [53]:
# menyimpan data train dan test
#df_train80.to_csv('df_train80.csv')
#df_test20.to_csv('df_test20.csv')

# Feature Extraction
TF-IDF

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Membuat objek vectorizer
tfidf_vect_8020 = TfidfVectorizer()

# Menghitung bobot TF-IDF
tfidf_vect_8020.fit(reviews['Reviews'])

# representasi vektor TF-IDF untuk data train & test
train_X_tfidf_8020 = tfidf_vect_8020.transform(df_train80['Reviews'])
test_X_tfidf_8020 = tfidf_vect_8020.transform(df_test20['Reviews'])

In [113]:
print(train_X_tfidf_8020)
#print(test_X_tfidf_8020)
# indeks kalimat, indeks kata/fitur, nilai tfidf

  (0, 2043)	0.3716004489097684
  (0, 1919)	0.3716004489097684
  (0, 1725)	0.20474007466736982
  (0, 1519)	0.22505780396306832
  (0, 1466)	0.22673585241912458
  (0, 1427)	0.1974020011620077
  (0, 1360)	0.3716004489097684
  (0, 952)	0.33496478767309334
  (0, 938)	0.12689327874717454
  (0, 887)	0.12689327874717454
  (0, 709)	0.2498994165728818
  (0, 614)	0.16699165471381475
  (0, 424)	0.17404933452822524
  (0, 381)	0.2321154837774788
  (0, 176)	0.22343139604535153
  (0, 161)	0.13435767267387225
  (0, 124)	0.1261254092591891
  (1, 1887)	0.2648980926621023
  (1, 1832)	0.363095574144571
  (1, 938)	0.13157721405840378
  (1, 814)	0.22252337605745093
  (1, 703)	0.3853171128773299
  (1, 662)	0.2333652273779101
  (1, 440)	0.6057721280739146
  (1, 173)	0.1552892170508488
  :	:
  (661, 458)	0.18377632813194925
  (661, 204)	0.131888045516611
  (661, 173)	0.1329345332267191
  (662, 2000)	0.31888802947883726
  (662, 1914)	0.28063269695860393
  (662, 1900)	0.3880289930042312
  (662, 1869)	0.21037673028

In [114]:
# shape (baris, fitur)
print(train_X_tfidf_8020.shape)
print(test_X_tfidf_8020.shape)

(664, 2086)
(167, 2086)


# Feature Selection
Chi-Square = [10%, 20%, 30%]

In [92]:
from sklearn.feature_selection import SelectKBest, chi2

percent_kept = 0.2  # persentase jumlah fitur terbaik yang dipertahankan

# jumlah fitur yang dipertahankan
num_features = int(percent_kept * train_X_tfidf_8020.shape[1])

# objek selector
selector = SelectKBest(chi2, k=num_features)

# memilih fitur terbaik dari train & test berdasarkan skor chi-square.
train_X_tfidf_8020_selected = selector.fit_transform(train_X_tfidf_8020, train_Y)
test_X_tfidf_8020_selected = selector.transform(test_X_tfidf_8020)

In [ ]:
print(train_X_tfidf_8020_selected.shape)
print(test_X_tfidf_8020_selected.shape)

# Proses Pelatihan
Chi-square = 0.1, 0.2, atau 0.3

C= 0.1, 1, 10, atau 100

kernel: rbf atau polynomial

nilai gamma = 0.0001, 0.001, 0.1, atau 1

In [93]:
# mengimpor kelas SVC dari modul svm dalam library scikit-learn
from sklearn.svm import SVC

# objek model dengan parameternya
model = SVC(C=1, kernel='rbf', gamma="scale")

# fit --> train/melatih
model.fit(train_X_tfidf_8020_selected, train_Y)

SVC(C=1)

# Proses Pengujian

In [94]:
from sklearn.metrics import accuracy_score

# Menggunakan model SVM yang telah dilatih sebelumnya, predict() --> prediksi pada data uji test_X_tfidf_8020.
predictions_SVM_8020 = model.predict(test_X_tfidf_8020_selected)

# Data frame untuk menampilkan hasil
test_prediction_8020 = pd.DataFrame()
test_prediction_8020['Reviews'] = test_X
test_prediction_8020['Label'] = predictions_SVM_8020

# Menghitung akurasi prediksi membandingkan hasil prediksi predictions_SVM_8020 dengan label sebenarnya test_Y menggunakan fungsi accuracy_score.
SVM_accuracy_8020 = accuracy_score(predictions_SVM_8020, test_Y)*100
# Membulatkan nilai akurasi menjadi 1 angka desimal menggunakan fungsi round.
SVM_accuracy_8020 = round(SVM_accuracy_8020,1)

In [95]:
# Menampilkan hasil test
test_prediction_8020

,Reviews,Label
610,kecewa sih sen nya warna hitam pas dateng nya ...,0
818,produk bagus sesuai sama gambar banget gitu ha...,1
290,bagus cuman susah atur jam nya susah tekan tekan,0
559,warna beda jauh digmbr jatuh nya wrnanya gelap,0
168,bagus kacamata mewah kayak mahal banget tebel ...,1
...,...,...
192,bagus cuma kirim lama bagus,1
650,kecewa ukur kaya ukur kecil bgt bahan tipis st...,0
456,jual gak amanah pesan datang cuma responnya la...,0
773,bagus,1


In [79]:
#test_prediction_8020.to_csv("test_prediction_8020.csv")

In [96]:
# Menampilkan akurasi
SVM_accuracy_8020

92.8

# Accuracy, Precision, Recall, f1-score

In [81]:
from sklearn.metrics import classification_report

print ("\nHere is the classification report:")
print (classification_report(test_Y, predictions_SVM_8020, zero_division='warn'))


Here is the classification report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.93        84
           1       1.00      0.86      0.92        83

    accuracy                           0.93       167
   macro avg       0.94      0.93      0.93       167
weighted avg       0.94      0.93      0.93       167



# Test the SVM classifier

In [66]:
review = "barangnya bagus banget"
review_vector = tfidf_vect_8020.transform([review]) # vectorizing
#print(model.predict(review_vector))

if(model.predict(review_vector)==1):
  print("Positif")
else:
  print("Negatif")

Positif


# Save the trained model and vectorizer

In [85]:
import joblib
#menyimpan model svm
joblib.dump(model, "model.pkl")
#menyimpan objek TfidfVectorizer
joblib.dump(tfidf_vect_8020, "vectorizer.pkl")

['vectorizer.pkl']

# Mencari Parameter Terbaik
Iterasi proses feature selection sampai evaluasi untuk mencari kombinasi parameter terbaik

In [75]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, chi2

chi = [0.1, 0.2, 0.3]
c = [0.1, 1, 10, 100]
kernel = ['rbf', 'poly']
gamma = [0.0001, 0.001, 0.1, 1, 'scale', 'auto']

def mulai(h, i, j, k):
    percent_kept = h
    num_features = int(percent_kept * train_X_tfidf_8020.shape[1])
    selector = SelectKBest(chi2, k=num_features)
    train_X_tfidf_8020_selected = selector.fit_transform(train_X_tfidf_8020, train_Y)
    test_X_tfidf_8020_selected = selector.transform(test_X_tfidf_8020)

    print(f"Chi Square = {h}\tParameter: c={i}, kernel={j}, gamma={k}")
    model = SVC(C=i, kernel=j, gamma=k)
    model.fit(train_X_tfidf_8020_selected, train_Y)

    predictions_SVM_8020 = model.predict(test_X_tfidf_8020_selected)
    test_prediction_8020 = pd.DataFrame()
    test_prediction_8020['Reviews'] = test_X
    test_prediction_8020['Label'] = predictions_SVM_8020
    SVM_accuracy_8020 = accuracy_score(predictions_SVM_8020, test_Y) * 100
    SVM_accuracy_8020 = round(SVM_accuracy_8020, 1)

    print(f"Akurasi: {SVM_accuracy_8020}")
    print("Here is the classification report:")
    print(classification_report(test_Y, predictions_SVM_8020, zero_division=0.0), end="\n")
    print("============================================================")

    data.append({'Chi Square': h, 'C': i, 'Kernel': j, 'Gamma': k, 'Accuracy': SVM_accuracy_8020})

for h in chi:
  data = []
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
  print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
  for i in c:
    for j in kernel:
      for k in gamma:
        mulai(h, i, j, k)
  df = pd.DataFrame(data)

  # Sort the DataFrame by Accuracy in descending order
  df_sorted = df.sort_values('Accuracy', ascending=False)

  # Count the number of combinations with the same accuracy
  accuracy_counts = df_sorted['Accuracy'].value_counts().reset_index()
  accuracy_counts.columns = ['Accuracy', 'Count']
  accuracy_counts_sorted = accuracy_counts.sort_values('Accuracy', ascending=False)

  print("All Combinations:")
  print(df)
  print("\nWorst Combination:")
  print(df_sorted.tail(1))
  print("\nModerate Combination:")
  print(df_sorted.iloc[len(df_sorted) // 2])
  print("\nBest Combination:")
  print(df_sorted.head(1))
  print("\nAccuracy Counts:")
  print(accuracy_counts_sorted)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Chi Square = 0.1	Parameter: c=0.1, kernel=rbf, gamma=0.0001
Akurasi: 50.3
Here is the classification report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        84
           1       0.00      0.00      0.00        83

    accuracy                           0.50       167
   macro avg       0.25      0.50      0.33       167
weighted avg       0.25      0.50      0.34       167

Chi Square = 0.1	Parameter: c=0.1, kernel=rbf, gamma=0.001
Akurasi: 50.3
Here is the classification report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        84
           1       0.00      0.00      0.00        83

    accuracy                           0.50       167
   macro avg       0.25      0.50      0.33       167
weighted avg       0.25      0.50      0.34       167

Chi Square = 0.1	Parameter: c=0.1